In [ ]:
language = 'pt'

# 1. Gravação de Áudio Com Python (e Uma Pitada de JavaScript) 🎤

In [ ]:
# Referência: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be

from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

# Código JavaScript para gravar áudio do usuário usando a "MediaStream Recording API"
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  # Executa o código JavaScript para gravar o áudio
  display(Javascript(RECORD))
  # Recebe o áudio gravado como resultado do JavaScript
  js_result = output.eval_js('record(%s)' % (sec * 1000))
   # Decodifica o áudio em base64
  audio = b64decode(js_result.split(',')[1])
  # Salva o áudio em um arquivo
  file_name = 'request_audio.wav'
  with open(file_name, 'wb') as f:
    f.write(audio)
  # Retorna o caminho do arquivo de áudio (pasta padrão do Google Colab)
  return f'/content/{file_name}'

# Grava o áudio do usuário por um tempo determinado (padrão 5 segundos)
print('Ouvindo...\n')
record_file = record()

# Exibe o áudio gravado
display(Audio(record_file, autoplay=False))

Ouvindo...



<IPython.core.display.Javascript object>

# 2. Reconhecimento de Fala com Whisper (OpenAI) 🧠

In [ ]:
!pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.4 MB/s eta 0:00:00


In [ ]:
import whisper

# Selecione o modelo do Whisper que melhor atenda às suas necessidades:
# https://github.com/openai/whisper#available-models-and-languages
model = whisper.load_model("small")

# Transcreve o audio gravado anteriormente.
result = model.transcribe(record_file, fp16=False, language=language)
transcription = result["text"]
print(transcription)

 e é o Github.


# 3. Integração com a API do GEMINI 💬

In [ ]:
!pip install -q -U google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 15.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 which is incompatible.


In [ ]:
import os

# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto "TODO" por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
os.environ['GOOGLE_API_KEY'] = 'TODO'

In [ ]:
import google.generativeai as genai
from google.colab import userdata

# 1. Configuração Inicial
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=api_key)

    # 2. TRUQUE: Listar modelos disponíveis e escolher o primeiro que funcione
    # Isso evita o erro 404 de 'model not found'
    available_models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]

    if not available_models:
        raise Exception("Nenhum modelo de chat disponível na sua chave.")

    # Escolhemos o modelo mais moderno da lista automaticamente
    selected_model = available_models[0]
    model = genai.GenerativeModel(selected_model)

    # 3. Gera a resposta usando a 'transcription' da Parte 2
    response = model.generate_content(transcription)
    chatgpt_response = response.text

    print("-" * 30)
    print(f"Modelo usado: {selected_model}")
    print("Resposta do Assistente:", chatgpt_response)
    print("-" * 30)

except Exception as e:
    print(f"Erro na Integração: {e}")
    chatgpt_response = "Tive um problema técnico, mas você pode tentar gravar de novo."

------------------------------
Modelo usado: models/gemini-2.5-flash
Resposta do Assistente: O GitHub é uma **plataforma online de hospedagem de código-fonte e colaboração para desenvolvedores**, amplamente utilizada para projetos de software. Ele é construído sobre o sistema de controle de versão **Git**.

Para entender o GitHub, é importante primeiro entender o **Git**:

*   **Git:** É um sistema de controle de versão distribuído (VCS - Version Control System). Ele permite que os desenvolvedores rastreiem as alterações em seu código ao longo do tempo, voltem a versões anteriores se necessário, e trabalhem em um mesmo projeto simultaneamente sem sobrescrever o trabalho uns dos outros. Pense nele como uma máquina do tempo e um sistema de gerenciamento de documentos para código.

Agora, o **GitHub** entra em cena:

1.  **Hospedagem de Código:** O GitHub pega o poder do Git e o coloca em uma interface web amigável. Ele permite que você armazene seus repositórios Git (onde seu código e to

# 4. Sintetizando a Resposta do GEMINI Como Voz (gTTS) 🔊

In [ ]:
!pip install gTTS

In [ ]:
from gtts import  gTTS

# Cria um objeto gTTS com a resposta gerada pelo ChatGPT e a língua que será sintetizada em voz (variável "language").
gtts_object = gTTS(text=chatgpt_response, lang=language, slow=False)

# Salva o áudio da resposta no arquivo especificado (pasta padrão do Google Colab)
response_audio = "/content/response_audio.wav"
gtts_object.save(response_audio)

# Reproduz o áudio da resposta salvo no arquivo
display(Audio(response_audio, autoplay=True))